In [7]:
import requests
import json
from bs4 import BeautifulSoup
import torch
import pandas as pd
import time

In [8]:
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer, util

# # Загрузка данных из JSON
# with open('../data/movies_data.json', 'r', encoding='utf-8') as f:
#     movies_data = json.load(f)

# # Инициализация модели rubert-tiny2
# model = SentenceTransformer("cointegrated/rubert-tiny2")

# # Извлечение названий и описаний
# titles = [movie['movie_title'] for movie in movies_data]
# descriptions = [movie['description'] for movie in movies_data]

# # Генерация эмбеддингов для названий и описаний
# title_embeddings = model.encode(titles).astype("float32")
# description_embeddings = model.encode(descriptions).astype("float32")

# # Создание FAISS индексов для быстрого поиска
# title_index = faiss.IndexFlatIP(title_embeddings.shape[1])
# title_index.add(title_embeddings)

# description_index = faiss.IndexFlatIP(description_embeddings.shape[1])
# description_index.add(description_embeddings)

# # Функция комбинированного поиска
# def search_movies(query, search_by="title", threshold=5, top_k=7):
#     """
#     Параметры:
#     query (str): запрос для поиска
#     search_by (str): искать по названию или описанию ("title" или "description")
#     threshold (int): порог длины запроса для выбора симметричного или асимметричного поиска
#     top_k (int): количество выводимых результатов (топ-K)

#     Возвращает:
#     Список найденных фильмов и их сходство.
#     """
#     if search_by == "title":
#         embeddings = title_embeddings
#         index = title_index
#         corpus = titles
#     else:
#         embeddings = description_embeddings
#         index = description_index
#         corpus = descriptions

#     # Определение типа поиска на основе длины запроса
#     if len(query.split()) > threshold:
#         # Длинный запрос: симметричный поиск
#         query_embedding = model.encode([query])
#         similarities = util.cos_sim(query_embedding, embeddings)
#         results = [(corpus[i], similarities[0][i].item()) for i in range(len(corpus))]
#     else:
#         # Короткий запрос: асимметричный поиск с FAISS
#         query_embedding = model.encode([query]).astype("float32")
#         D, I = index.search(query_embedding, k=top_k)
#         results = [(corpus[i], D[0][idx]) for idx, i in enumerate(I[0])]

#     # Сортировка и выбор топ-K результатов
#     results = sorted(results, key=lambda x: x[1], reverse=True)[:top_k]
#     return results

# # Пример поиска по названию
# query = "Человек-паук"
# results_by_title = search_movies(query, search_by="title")
# print("Результаты поиска по названию:")
# for title, similarity in results_by_title:
#     print(f"Название: {title}, Сходство: {similarity:.4f}")

# # Пример поиска по описанию
# query = "фильм о супергерое, который сражается со злодеями"
# results_by_description = search_movies(query, search_by="description")
# print("\nРезультаты поиска по описанию:")
# for description, similarity in results_by_description:
#     print(f"Описание: {description[:100]}..., Сходство: {similarity:.4f}")


In [9]:
movies_data = pd.read_csv('../data/all_data.csv')
movies_data = movies_data.rename(columns={'film_name': 'movie_title', 'image': 'image_url'})

In [10]:
movies_data

,movie_title,IMDb,director,actors,image_url,description
0,Человек-паук: Нет пути домой,8.2,Джон Уоттс,"Джон Уоттс, Том Холланд, Зендая, Бенедикт Камб...",https://resizer.mail.ru/p/77287c77-f712-5110-8...,"Теперь, когда весь мир узнал, кто скрывается з..."
1,Джентльмены,7.8,Гай Ричи,"Гай Ричи, Мэттью Макконахи, Чарли Ханнэм, Хью ...",https://resizer.mail.ru/p/269f49ca-a456-59bf-a...,"Талантливый выпускник Оксфорда, применив свой ..."
2,Выживший,8.0,Алехандро Гонсалес Иньярриту,"Алехандро Гонсалес Иньярриту, Леонардо ДиКапри...",https://resizer.mail.ru/p/e8c2b865-d24d-585b-b...,Легендарный первопроходец Хью Гласс серьезно р...
3,Гнев человеческий,7.1,Гай Ричи,"Гай Ричи, Джейсон Стэйтем, Алекс Фернс, Холт М...",https://resizer.mail.ru/p/61fc89c3-5ccc-598a-9...,Загадочный и холодный на вид Эйч (Джейсон Стэй...
4,1+1,8.5,Оливье Накаш,"Оливье Накаш, Эрик Толедано, Франсуа Клюзе, Ом...",https://resizer.mail.ru/p/25e85ca5-28f1-5cd6-8...,В результате несчастного случая богатый бизнес...
...,...,...,...,...,...,...
5747,Киевский торт,Рейтинг не найден,Алексей Шапарев,"Алексей Шапарев, Дмитрий Ступка, Диана Сиротин...",https://resizer.mail.ru/p/3e782fa8-c757-5e5c-9...,В Киев приезжает провинциалка из Гайсина. Стол...
5748,Обыкновенное сердце,7.9,Райан Мерфи,"Райан Мерфи, Марк Руффало, Мэтт Бомер, Тейлор ...",https://resizer.mail.ru/p/85140383-8a43-5e92-9...,Гомосексуальный активист пытается поднять проб...
5749,Святой Винсент,7.2,Тед Мелфи,"Тед Мелфи, Билл Мюррей, Мелисса МакКарти, Наом...",https://resizer.mail.ru/p/748a2567-edc2-5802-b...,"Фильм расскажет о 12-летнем мальчике, который,..."
5750,Узник старой усадьбы,Рейтинг не найден,Марина Сулейманова,"Марина Сулейманова, Сергей Шнуров, Евгений Ток...",https://resizer.mail.ru/p/c0d4eed9-a68e-5dca-8...,Молодой парень Вениамин решает подработать лет...


In [11]:
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import re


model = SentenceTransformer("cointegrated/rubert-tiny2")

def find_movies_with_all_words(query, df):
    query_words = set(query.lower().split())
    matched_indices = set()
    
    def contains_all_words(text):
        if pd.isna(text):
            return False
        # Используем регулярное выражение для разделения по запятой и пробелу
        text_words = set(re.split(r'[,\s]+', text.lower()))
        return query_words.issubset(text_words)
    
    # Список столбцов, по которым будем искать
    columns_to_search = ['movie_title', 'director', 'actors']
    
    for column in columns_to_search:
        matching_texts = df[df[column].apply(contains_all_words)]
        matched_indices.update(matching_texts.index)
    
    if matched_indices:
        # Преобразуем множество в список перед выбором
        return df.loc[list(matched_indices)].drop_duplicates()
    else:
        return df
# def find_movies_with_all_words(query, df):
#     query_words = set(query.lower().split())
    
#     def contains_all_words(text):
#         text_words = set(text.lower().split())
#         return query_words.issubset(text_words)

#     # Список столбцов, по которым будем искать
#     columns_to_search = ['movie_title', 'director', 'actors', 'description']
    
#     for column in columns_to_search:
#         matching_texts = df[df[column].apply(contains_all_words)]
#         if not matching_texts.empty:
#             return matching_texts
    
#     # Если ни в одном из столбцов не найдено совпадений, возвращаем исходный DataFrame
#     return df

# titles = movies_data['movie_title'].tolist()
# descriptions = movies_data['description'].tolist()
# images = movies_data['image_url'].tolist()
# actors = movies_data['actors'].tolist()
# directors = movies_data['director'].tolist()

# corpus = [(image, title, description) for image, title, description in zip(images, titles, descriptions)]
# titles = [(movie['image_url'], movie['movie_title']) for movie in movies_data]
# descriptions = [(movie['image_url'], movie['movie_title'], movie['description']) for movie in movies_data]

# Генерация эмбеддингов для названий и описаний
# title_texts = [movie[1] for movie in titles] 
# description_texts = [movie[2] for movie in descriptions] 

# Генерация эмбеддингов для названий и описаний
# title_embeddings = model.encode(titles).astype("float32")
# description_embeddings = model.encode(descriptions).astype("float32")

# # Создание FAISS индексов для быстрого поиска
# title_index = faiss.IndexFlatIP(title_embeddings.shape[1])
# title_index.add(title_embeddings)

# description_index = faiss.IndexFlatIP(description_embeddings.shape[1])
# description_index.add(description_embeddings)

def search_movies(query, movies_data, threshold=5, top_k=7):
    if len(movies_data) < top_k:
        top_k = len(movies_data)
    
    titles = movies_data['movie_title'].tolist()
    descriptions = movies_data['description'].tolist()
    images = movies_data['image_url'].tolist()
    actors = movies_data['actors'].tolist()
    directors = movies_data['director'].tolist()

    corpus = [(image, title, description) for image, title, description in zip(images, titles, descriptions)]
    # titles = [(movie['image_url'], movie['movie_title']) for movie in movies_data]
    # descriptions = [(movie['image_url'], movie['movie_title'], movie['description']) for movie in movies_data]

    # Генерация эмбеддингов для названий и описаний
    # title_texts = [movie[1] for movie in titles] 
    # description_texts = [movie[2] for movie in descriptions] 

    # Генерация эмбеддингов для названий и описаний
    title_embeddings = model.encode(titles).astype("float32")
    description_embeddings = model.encode(descriptions).astype("float32")

    # Создание FAISS индексов для быстрого поиска
    title_index = faiss.IndexFlatIP(title_embeddings.shape[1])
    title_index.add(title_embeddings)

    description_index = faiss.IndexFlatIP(description_embeddings.shape[1])
    description_index.add(description_embeddings)


    embeddings = description_embeddings
    index = description_index

    if len(query.split()) > threshold:
        # Длинный запрос: симметричный поиск
        query_embedding = model.encode([query])
        similarities = util.cos_sim(query_embedding, embeddings)
        results = [(corpus[i], similarities[0][i].item()) for i in range(len(corpus))]
    else:
        # Короткий запрос: асимметричный поиск с FAISS
        query_embedding = model.encode([query]).astype("float32")
        D, I = index.search(query_embedding, k=top_k)
        results = [(corpus[i], D[0][idx]) for idx, i in enumerate(I[0])]

    # Сортировка и выбор топ-K результатов
    results = sorted(results, key=lambda x: x[1], reverse=True)[:top_k]
    
    # Возвращаем image_url, title, description и сходство
    formatted_results = [
        {"image_url": res[0][0], "movie_title": res[0][1], "description": res[0][2], "similarity": res[1]}
        for res in results
    ]
    return formatted_results

In [15]:
txt = 'том холланд'
if txt:        
        start_time = time.time()
        movies_data = find_movies_with_all_words(txt, movies_data)
        movies_data = movies_data.drop_duplicates(subset=['movie_title'])
        results = search_movies(txt, movies_data)
        end_time = time.time()